In [164]:
from random import random
import numpy as np
from itertools import combinations
import math
import random

In [165]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

In [166]:
# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g


In [167]:
### create a graph with planted clique of size = clique_size

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clique_size: clique size

def generate_graph(num_vertices, pr, clique_size):
    def plant_clique():
        for i in range(clique_size):
            for j in range(i + 1, clique_size):
                for l in range(j + 1, clique_size):
                    a = clique_vertices[i]
                    # print(a)
                    vec[a] += 1
                    b = clique_vertices[j]
                    # print(b)
                    vec[b] += 1
                    c = clique_vertices[l]
                    # print(c)
                    vec[c] += 1
                    add_all(a, b, c, g)

    g = np.array([np.array([np.array([0 for i in range(0, num_vertices)]) for i in range(num_vertices)]) for i in
                  range(num_vertices)])
    vec = np.array([0 for i in range(0, num_vertices)])

    # Set edges
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                a = np.random.uniform(0, 1, 1)
                # every edge is included independently with probability 1/2
                if a < pr:
                    vec[i] += 1
                    vec[j] += 1
                    vec[k] += 1
                    add_all(i, j, k, g)

    # clique_vertices = random.sample(range(num_vertices), clique_size) # is is wrong??; would violate the clique size constraint
    
    clique_vertices = np.random.choice(range(num_vertices), clique_size, replace=False)

    plant_clique()

    return g, vec, clique_vertices


In [168]:
# lst = np.random.choice(range(10), 5, replace=False)
# lst

In [169]:
# calculate the total number of edges on a graph with num_vertices vertices
def find_num_edges(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [170]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    # active_count: number of vertices that are associated with at least 1 edge
    active_count = np.count_nonzero(vec)
    # number of edges
    edge_sum = find_num_edges(g, num_vertices)
    if edge_sum == 6 * math.comb(active_count, 3):
        return True
    return False

In [171]:
# remove edge
def remove_edges(g, num_vertices, vec, curr):
    vec[curr] = 0
    graph_copy = g.copy()
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if graph_copy[curr, j, k] == 1:
                vec[j] -= 1
                vec[k] -= 1
                remove_all(curr, j, k, graph_copy)
    return graph_copy, vec

### Generate a graph

In [172]:
# driver program

n = 50 # number of vertices
p = 0.5 # probablity of edge being included
k = 10 # clique size

# generate graph
res = generate_graph(n, p, k)

# G graph with planted clique
# V: vector storing the number of edges associated with each vertice in the graph
# clique_vertices: the set of clique vertices
G, V, clique_vertices = res[0],  res[1], res[-1]

# print(f"graph: {G}\n")
G_0 = G.copy()
print(f"edge-occurence vector: {V}\n")
print(f"clique vertices: {clique_vertices}\n")



edge-occurence vector: [21 15 17 23 17 17 24 14 17 21]

clique vertices: [6 0 1 3 9]



### Removal Phase

In [173]:
clique = False
iter = 0
removed = []
while not clique:
    
    iter += 1

    curr = -1
    idx_sorted = np.argsort(V)
    
    print(f"i = {iter}\nindex sorted: {idx_sorted}")
    print(f"number of edges associated with each vertex: {V}")
    # print(f'current set of clique vertices: {clique_vertices}')

    for idx in range(n):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break
    print(f"vertex removed: {curr} number of edges: {V[curr]}\n")
    A = remove_edges(G, n, V, curr)
    # print(A[1])
    G = A[0]

    clique = is_clique(G, V, n)

i = 1
index sorted: [7 1 2 4 5 8 0 9 3 6]
number of edges associated with each vertex: [21 15 17 23 17 17 24 14 17 21]
vertex removed: 7 number of edges: 14

i = 2
index sorted: [7 4 1 2 5 8 3 0 9 6]
number of edges associated with each vertex: [19 13 13 18 12 14 22  0 14 19]
vertex removed: 4 number of edges: 12

i = 3
index sorted: [4 7 2 5 8 1 0 3 9 6]
number of edges associated with each vertex: [15 12  9 16  0  9 20  0 11 16]
vertex removed: 2 number of edges: 9

i = 4
index sorted: [2 4 7 5 8 1 0 3 9 6]
number of edges associated with each vertex: [13 11  0 13  0  6 16  0  8 14]
vertex removed: 5 number of edges: 6

i = 5
index sorted: [2 4 5 7 8 1 3 9 0 6]
number of edges associated with each vertex: [13 10  0 10  0  0 13  0  5 12]
vertex removed: 8 number of edges: 5

i = 6
index sorted: [2 4 5 7 8 1 3 0 9 6]
number of edges associated with each vertex: [10  8  0  9  0  0 11  0  0 10]
vertex removed: 1 number of edges: 8

i = 7
index sorted: [1 2 4 5 7 8 3 0 9 6]
number of edge

In [174]:
removed

[7, 4, 2, 5, 8, 1, 3, 0]

In [175]:
# clique_v = res[-1]
# clique_v

### Inclusion Phase

In [176]:
# clique_vertices

In [177]:
full_vertices = np.arange(n)
full_vertices

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [178]:
included = np.setdiff1d(full_vertices, removed)
included

array([6, 9])

In [179]:
def inclusion_phase(removed, clique_vertices, G_0):
    def connected(idx, clique_vertices):
        for j in range(len(clique_vertices)):
                for k in range(j+1, len(clique_vertices)):
                    if G_0[idx, clique_vertices[j], clique_vertices[k]] != 1:
                        print(f" vertex {idx} is not connected to all clique vertices")  
                        return False
        return True
                        
    for idx in removed:
        if connected(idx, clique_vertices):
            print(f"add {idx} to clique!")
            clique_vertices = np.append(clique_vertices, idx)

    return clique_vertices
    

In [180]:
res = inclusion_phase(removed, included, G_0)
res

 vertex 7 is not connected to all clique vertices
 vertex 4 is not connected to all clique vertices
add 2 to clique!
 vertex 5 is not connected to all clique vertices
add 8 to clique!
 vertex 1 is not connected to all clique vertices
 vertex 3 is not connected to all clique vertices
 vertex 0 is not connected to all clique vertices


array([6, 9, 2, 8])